In [1]:
import requests
from lxml import etree
import time
import re
import csv

In [2]:
headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Referer': 'https://www.lagou.com/jobs/list_%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90/p-city_0?&cl=false&fromSearch=true&labelWords=&suginput=',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
    }

In [3]:
def save_cvs(data):
    #try:
        with open('positiondata.csv', 'a', encoding='utf-8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(data)
    #except:
        #print('write csv error!')

In [4]:
def request_list_page():
    url = 'https://www.lagou.com/jobs/positionAjax.json?needAddtionalResult=false'
    data = {
        'first': "false",
        'pn': 1,
        'kd': '数据分析'
    }
    for x in range(1,30):
        url_first='https://www.lagou.com/jobs/list_%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90/p-city_0?&cl=false&fromSearch=true&labelWords=&suginput='
        s = requests.Session()
        s.get(url_first, headers=headers, timeout=10)  # 请求首页获取cookies
        cookie = s.cookies  # 为此次获取的cookies
        
        data['pn'] = x
        response = s.post(url, data=data, headers=headers, cookies=cookie,timeout=3)
        time.sleep(3)
        # json方法：如果返回来的是json数据。那么这个方法会自动的load成字典
        result = response.json()
        positions = result['content']['positionResult']['result']
        for position in positions:
            positionId = position['positionId']
            position_url = 'https://www.lagou.com/jobs/%s.html' % positionId
            parse_postion_detail(position_url)
            time.sleep(10)
            #break
        #break

In [5]:
def parse_postion_detail(url):
    response = requests.get(url,headers=headers)
    text = response.text
    html = etree.HTML(text)
    try:
        domain= html.xpath("//h4[@class='c_feature_name']/text()")[0].replace('\n','').replace(' ','')
        stage=html.xpath("//h4[@class='c_feature_name']/text()")[1].replace('\n','').replace(' ','')
        scale= html.xpath("//h4[@class='c_feature_name']/text()")[2].replace('\n','').replace(' ','')
        print(domain,stage,scale)
        position= html.xpath("//h1[@class='name']/text()")[0]
        company = html.xpath("//h4[@class='company']/text()")[0]
        job_request_spans = html.xpath("//dd[@class='job_request']//span")
        salary = job_request_spans[0].xpath('.//text()')[0].strip()
        city = job_request_spans[1].xpath(".//text()")[0].strip()
        city = re.sub(r"[\s/]","",city)
        work_years = job_request_spans[2].xpath(".//text()")[0].strip()
        work_years = re.sub(r"[\s/]","",work_years)
        education = job_request_spans[3].xpath(".//text()")[0].strip()
        education = re.sub(r"[\s/]","",education)
        desc = "".join(html.xpath("//dd[@class='job_bt']//text()")).replace('\n','').replace(' ','')
        res=[position,company,salary,city,work_years,education,desc,domain,stage,scale]
        save_cvs(res)
        print(res)
        print('====')
    except:
        pass

In [6]:
def main():
    #首行写入cvs文件
    data=['position','company','salary','city','work_years','education','desc','domain','stage','scale']
    save_cvs(data)
    request_list_page()

In [7]:
if __name__ == '__main__':
    main()

['数据挖掘工程师', '广州思迈特研发部招聘', '20k-30k', '广州', '经验3-5年', '硕士及以上', '职位描述：岗位职责：1、负责数据科学解决方案的规划；2、协助软件工程师开发相应的算法处理代码；3、针对实际问题进行新算法的建模、仿真与测评；4、完成项目产品的相关文档的设计，包括需求文档、概要设计文档和详细设计文档；5、完成相应的技术攻关难点的工作。岗位要求：1、大学本科及以上学历，应用数学、统计及相关专业，3年以上软件开发工作经验；2、精通Matlab、python、R语言其中一种，熟悉相关开发工具的使用，有一定的项目开发经验；3、有较强的分析和解决问题的能力；4、具备良好的学习能力和口头表达能力，具备良好的英语阅读能力，有一定的实验报告编写能力；5、具备数据科学比赛经验者优先。']
====
['资深数据分析师', '快手X3招聘', '20k-40k', '北京', '经验3-5年', '本科及以上', '职位描述：职位描述1、以一定定量方法解决产品、运营、战略相关的各种复杂商业分析问题，能独立完成方法或策略设计、数据处理与分析的工程实现，结论展示的整个流程；2、对公司内外部数据进行挖掘，在深入理解用户行为、产品特点的基础上，自主发掘课题，提出建议，数据驱动产品、运营与战略决策。任职要求1、很强的商业洞察能力，数字敏感性与逻辑思维能力；2、对统计学、运筹学、机器学习等各种定量方法有较深入理解，能以合适的分析框架解决各种复杂问题，包括策略设计、数学建模；3、熟练使用HQL，Python，R等分析工具和办公软件，快速学习能力强；4、有音乐类视频类项目经验优先。']
====
['数据分析', '快手运营分析组招聘', '8k-10k', '成都', '经验1-3年', '本科及以上', '职位描述：岗位职责:1、负责团队业务、人员指标建立及数据体系搭建，以衡量团队运营能力；2、负责团队数据报表模板建立，并进行常态数据统计、监控，推动数据平台的建立及完善；3、及时对数据进行深入分析，发现问题并提出改进意见，参与推动业务团队改善；4、根据团队需要，提供专项数据支持。任职资格:1、2年以上数据分析工作经验，有客服或审核团队数据经验者优先；2、本科及以上学历，专业不限，统计学或数学相关专业优先；3、熟练运用Excel进行数据统计与分析；4、具备较强

['数据分析', '百弓网络招聘', '12k-20k', '上海', '经验3-5年', '本科及以上', '职位描述：岗位职责:1、挖掘营运业务分析主题，开展数据分析工作，基于数据分析成果，为管理层和业务部门提供商业策略分析和业务优化建议，并推进建议落地执行；2、优化和完善所属方向的数据分析体系，包括核心指标体系、报表体系、专题分析体系；3、推动数字化运营及业务；4、数据治理、数据架构等数据领域相关其它工作。岗位要求：1、3年以上数据分析应用工作经验，对金融数据分析有较为深刻的理解；2、深刻理解并熟练掌握统计学、数据挖掘算法原理，对常用数据挖掘算法有和工具有一定了解；3、了解当前大数据分析技术，有相关实施经验者优先；4、对数据治理、数据架构有一定理解；5、计算机、数学、统计学等相关专业，本科及以上学历。']
====
['数据分析', '圆通运营部招聘', '8k-15k', '杭州', '经验3-5年', '大专及以上', '职位描述：岗位职责：1、数据营运工作，运单数据、卸车数据、成本结算数据、时效数据的常见维度分析；内部数据库搭建：在平时数据处理过程中，积累和存储数据；支持部门内部数据输出；2、数据可视化平台筹建：梳理数据的整体框架，细化订单数据的分析维度、角度、指标，采用BI工具对其进行了效果展示；3、对部门及公司决策提供数据支持；参与各业务部门的重点项目，负责从数据的角度给出决策建议；4、通过专题分析，对业务问题进行深入分析，为业务运营决策提供数据支持；5、沉淀分析思路与框架，提炼数据产品需求，协作并推动数据产品落地；6、针对具体业务问题，研究影响用户的潜在因素，进行数据分析验证并提出改善举措；任职资格：1、本科及以上学历，数学、统计学、计算机、金融或相关专业；2、3年以上互联网数据分析工作经验，能快速理解业务，具有出色的逻辑分析能力和对数据的敏感度，善于发现问题，并具有独立问题解决能力3、熟悉SQL语言，熟练操作Excel，熟练使用1种以上数据分析和挖掘工具（如SPSS/Clementine、SAS/EM、R等）；4、良好的逻辑分析、文字表达及沟通推进能力，能独立完成专题分析及跟进落地；5、熟悉快递行业物流干线运输，零担、卡班等业务场景，有快递、物流行业、互联网+物流平台工作经验者优先。6、有数据模型建立和运营经验、数据化运营经验、数据类产品类规划经

['大数据分析运营', '美的IoT招聘', '15k-25k', '广州', '经验3-5年', '本科及以上', '职位描述：岗位要求：负责美的智能产品数据挖掘分析，驱动产品智能及用户体验提升，主要包括：1、通过数据洞察发现问题及分析场景，撰写分析报告及建议解决方案，推动业务变革；2、使用常用的数据分析及挖掘算法模型，按需求建立模型，为业务提供决策支持；3、面向用户、事业部及业务部门发布数据报告及数据小品；4、参与数据产品及数据可视化建设；5、挖掘智能用户、设备数据，制定标签为营销企划提供数据支撑；工作要求：1、统招本科及以上学历，3年以上工作经验，2年以上数据分析或数据挖掘经验；2、熟悉常用的数理统计理论、数据分析方法及挖掘算法；2、掌握R、Python、SPSS等至少一钟分析挖掘工具；2、熟练使用办公软件，有较强的报告撰写能力及一定的汇报能力；4、数据大数据体系基本架构，能够使用SQL查询取数"【加分项】1、有机器学习、深度学习实践经验；2、有用户标签应用产品经验；']
====
['数据分析', '微店招聘', '10k-18k', '北京', '经验应届毕业生', '本科及以上', '职位描述：岗位职责：1、负责数据采集、维护及数据异常的跟踪处理;2、负责市场、产品及运营项目的前期调研，可行性分析、目标梳理，实施跟踪及效果分析3、竞品相关的市场对比分析，整理编写数据分析报告，及时发现和分析其中隐含的变化和问题任职资格：1.\xa0\xa0本科及以上学历，数学、统计、经济学、计算机等相关专业优先；2.\xa0\xa0熟练使用Excel、PPT，具备数据库基础，会使用SQL；3.\xa0\xa0较强的数据敏感度、结构化思维和沟通协调能力；4.\xa0\xa0强烈的好奇心和探索精神，充满激情,乐于接受挑战，具备一定的抗压能力；5.\xa0\xa0具有很强的团队意识，有诚信、愿意分享和承担责任。']
====
['数据分析师', '美图公司招聘', '8k-16k', '厦门', '经验1-3年', '本科及以上', '职位描述：美图的数据科学团队是⼀个数据驱动决策的团队。我们正在寻找一位合格的具有大数据分析和技术经验的数据科学家或高级数据分析师。理想的候选人是数据驱动型的问题解决者，能够运用SQL分析大量数据，输出分析报告以推动业务决策，构建自动化数据模型，提

['数据分析工程师-Toc Group', 'ShopeeShopee dev招聘', '15k-30k', '深圳', '经验1-3年', '本科及以上', '职位描述：岗位职责：负责业务数据分析支持、数据提取支持深入业务场景，设计、优化数仓数据模型优化数据开发验证流程，提高数据服务质量负责数据体系建设、分析及开发，包括建立指标体系，完成数据分析，报表开发及可视化呈现，完成数据仓库、数据报表的迭代开发、优化负责各业务的海量数据分析和挖掘工作；任职要求：数学、统计学、计算机等相关专业优先有丰富的数据分析和数据挖掘经验至少熟练掌握一种查询工具（SQL、hive），了解Hadoop工作原理熟悉JAVA，C，C++等至少一门编程语言熟悉Hadoop/Spark/Hive/Storm等大数据处理系统，了解其运行原理掌握数据分析方法、数据建模知识，能将相关方法应用于实际工作具备良好的沟通技巧和表达能力，有较好的报告撰写能力数据敏感，有良好的逻辑思维能力，有强烈的责任感，较强的沟通能力和团队合作精神，勇于承担工作压力']
====
['资深数据分析师', '贝壳招聘', '35k-55k', '北京', '经验5-10年', '本科及以上', '职位描述：工作职责:1.根据业务需求，协助制定指标，梳理指标体系，设计数据埋点方案，并跟进落实;2.负责中心日常运营数据的统计、监控，并进行量化分析，定制化推送共性基础数据，实现常规报表的自动化；3.对产品数据进行例行监控，及时发现数据的异常波动，并通过深入分析了解其原因，定期输出相关数据分析报告；4.通过数据分析挖掘，分析功能使用场景、用户画像特征，发现潜在的缺陷与机会，为业务决策提供数据支撑；5.根据数据分析结论，推导解决方案，并推进方案应用落地。任职资格:1.本科及以上学历，数学、统计学、计算机等相关专业优先；2.有1年以上数据分析相关工作经验；3.熟练运用SQL，有python或R基础，有tableau相关工作经验优先；4.了解hadoop生态，Hive及相关hql，ETL等工作，熟悉数据仓库(DW)/商业智能(BI)的数据框架；5.拥有敏锐分析能力，严谨客观的分析态度，优秀的学习能力和沟通表达能力，能够快速熟悉业务，并对用户体验有敏锐的数据洞察力；6.性格稳重，良好的沟通能力，富有团队合作精神，抗压力强，对工作专注、投入。']

['数据分析', '光点科技招聘', '9k-13k', '上海', '经验1-3年', '本科及以上', '职位描述：岗位职责：1.利用机器学习方法解决实际业务问题；2.对基础数据进行整理清洗和统计分析，刷选变量，提取特征，并创建模型；3.参与各类模型的设计、开发、实施与跟踪，结合实际效果不断进行模型的训练和迭代；任职要求：1.熟练掌握Python的数据挖掘算法来实现机器学习或深度学习；2.本科以上学历，至少具备以下教育背景中的一种：计算机与信息科学类、数学类、统计类、计量经济学类、机器学习类等专业（硕士以上学历优先）；3.1年以上数据分析、数据挖掘及模型开发经验（保险金融公司、互联网公司大数据开发、建模与分析经验者优先）；4.熟悉主流机器学习库，熟悉主流数据库环境；5.对工作有热情，具备良好的学习能力、逻辑思维能力、沟通能力、团队协作能力和执行力。']
====
['数据分析', '易点租销售运营招聘', '15k-20k', '北京', '经验3-5年', '本科及以上', '职位描述：我司为B2B硬件IT产品租赁行业独角兽，为了更快的适应我司的业务，希望有相关B2B行业对销售数据分析有经验的优秀人员加入！岗位要求：1、定期追踪销售关键指标，并从业务层面分析存在的问题，洞察问题的原因，提出指导改进策略。2、销售业务关键业务流程分析，发现流程中的问题，提升销售效率；3、分析销售数据背后的真实销售数据以推动整个销售目标的达成；4、搭建销售数据体系，定期向部门提供日常报表、月报、季报等，针对指标数据异常，提出预警。任职资格：1、计算机、统计学、数学相关专业本科以上学历；2、具有较深的数据挖掘分析功底，两年以上数据挖掘相关工作经验，与销售运营、销售数据分析或业务管理等相关经验者优先考虑；3、熟悉常用数据分析工具，掌握sql语言、powerBI工具和Python分析；3、较强的组织、协调、沟通、人际交往能力以及敏锐的洞察力；4、责任心强，能承受较强的工作压力，具有很强的判断与决策能力，计划和执行能力及解决复杂问题的能力，能独立或支持团队完成数据分析工作；5、为人正直，踏实稳重，思维严谨，有良好的服务意识。']
====
['大数据分析师', '积微物联大数据中心招聘', '15k-20k', '成都', '经验3-5年', '本科及以上', '职位描述：岗位职责：1.\

...

['商业数据分析师', '轻轻家教电销运营招聘', '12k-20k', '上海', '经验1-3年', '本科及以上', '职位描述：【工作职责】1、业务经营结果导向，负责业务全链路的运营数据分析和策略制定；2、参与支持项目落地，搭建主题性的指标体系，监控项目进展，追踪效果；3、通过专题/项目分析，对业务进行深入分析，提供有效的策略建议，并能推动策略方案落地；4、根据业务需求，提供策略分析咨询与动作优化建议。【任职资格】1、熟练运用SQL，熟练操作excel，会hive，\xa0python是加分项；2、教育、互联网、咨询等相关行业数据分析2年以上工作经验；3、优秀的逻辑、沟通能力，具备自驱、细致、开放等特质，有一定的抗压能力；4、计算机相关、数学、统计学、财务学等相关本科及以上学历优先。']
====
['数据分析岗（业务方向）', '广发银行信用卡中心市场部招聘', '15k-25k', '广州', '经验3-5年', '本科及以上', '职位描述：岗位描述：1、通过数据分析、业务预测等方法，协助制定市场发卡、客户维护及品牌策略，完成市场和商户业务分析及规划的跟进执行。2、负责相关市场业务报表的开发及维护，并根据数据对市场活动、营销策略等进行评估及分析，提供合理建议。3、负责专项业务的分析与预测，KPI指标的制定与分解。4、负责市场关键指标及市场费用的跟踪管理，提供合理的业务预估，控制市场费用在预算范围内。5、参与制定市场相关业务策略，落实到业务规划中，并推动业务及策略的落地执行。岗位要求：1、重点本科及以上学历，理工科背景，3年以上相关工作经验；2、优秀的逻辑数理分析能力，有一定的市场敏锐度。']
====
['数据分析经理', '果壳市场部招聘', '10k-20k', '北京', '经验3-5年', '本科及以上', '职位描述：职位描述：岗位职责1、理解业务，构建业务指标体系，建立和完善日常业务报告体系；2、及时、准确、完整地汇总业务线和项目的运作情况，提供数据分析报告；3、通过专项分析，对业务问题进行深入分析，输出专项分析报告，为业务决策提供数据支持；4、参与数据仓库相关指标梳理，为数据仓库的构建提供支持。岗位要求1、1-5年数据分析工作经验，本科及以上学历，数学、计算机相关专业方向；2、熟悉数据库原理，具备数据的处理能力，掌握hive、SQL等相关

['中级数据分析师', '乐信圣文成都乐信招聘', '10k-20k', '成都', '经验1-3年', '本科及以上', '职位描述：岗位说明：1、基于公司产品的现有数据建立数据模型，设立指标，衡量产品改进；2、优化核心数据监控体系，进行每日的指标异动分析，及时发现业务问题并寻求解决方案3、根据用户行为分析数据，对产品优化提出建议并跟踪优化效果。任职条件：1、统计学，应用数学，计算机等相关专业本科及以上学历，1年以上互联网数据工作经验；2、精通Excel、SQL等，掌握第三方可视化数据平台优先；3、热爱数据分析工作、具有高度的数据敏感性与洞察力，高效的团队协作及沟通能力。']
====
['09216M-数据分析师', '平安健康保险健康险总部移动业务事业部招聘', '15k-20k', '北京', '经验3-5年', '本科及以上', '职位描述：工作职责1、梳理业务部门的分析需求，规划报表需求，推动团队实现日常数据可视化；2、基于业务特征优化并完善业务指标体系,包括KPI指标体系、用户画像标签体系等；3、负责对海量数据进行分析和挖掘，提炼出用户特征、行为路径，搭建相关数据模型4、跨部门沟通，负责所需数据的提取、查询、分析等工作，以满足业务需求；5、深入理解产品和业务方向及策略，持续监控运营和产品改进效果，为业务部门提供业务优化建议；6、为业务决策提供数据解决方案，并在数据营销方面提出策略建议;7、领导安排的其他工作。任职要求1、重点本科及以上学历，计算机、统计、金融，数据等相关专业；2、4年以上互联网或保险数据分析、挖掘工作经验；3、熟练使用EXCEL/SPSS/SAS/R/HIVE/SQL等工具，具备良好的逻辑思维和分析能力；4、熟悉数据收集，数据分析，数据建模和数据挖掘方法；5、能快速理解定位业务痛点，提出综合性解决方案；6、熟悉互联网行业，保险行业的优先。']
====
['数据分析师', 'NIO蔚来招聘', '18k-25k', '上海', '经验1-3年', '本科及以上', '职位描述：岗位内容：-跟随产品迭代，分析产品迭代中的数据需求，生成埋点规格，创建所需的中间表和可视化看板，形成产品优化的结论和建议，推动产品改进；-通过专项分析，输出专项分析报告，为业务发展提供策略和建议；-响应日常数据提取和输出；-数据产品的持续改进，推动实施数据输出自动

['BI工程师/数据分析师（天津）', '谷川联行信息管理事业部招聘', '8k-12k', '天津', '经验不限', '本科及以上', '职位描述：岗位职责\xa01.根据公司的战略目标，建立完善的数据体系，推进系统化及自动化;\xa02.针对不同业务场景，完善数据监测和分析体系，推动优化数据的准确率及可用性;\xa03.向各部门提供各类准确数据及分析报告，能够在数据分析基础上发现问题并提出针对性解决方案;\xa04.对核心指标进行日/周/月度监控，对异常波动进行分解，定位并解读具体原因;\xa05.协助搭建BI的多维分析报表体系，协助完成数据仓库建模;\xa06.负责用户行为数据埋点体系;\xa0任职要求\xa01.本科及以上学历，统计学、数学、计算机专业优先，1年及以上数据分析经验优先；2.对商业和数字敏感，具备良好的逻辑分析能力、严谨的推理能力和团队合作精神;\xa03.精通SQL，熟练掌握mysql/hive数据库的SQL处理技巧；4.至少熟练使用Shell、Python、Perl、Tcl等脚本语言之一；5.对数据敏感，有独立的分析思路和观点，并能够在海量数据中找到数据支持;\xa06.具有良好的沟通能力、判断能力、学习能力，做事谨慎、认真细心;\xa07.需具有tableau、FineBI等主流数据分析工具的使用经验;8.工作认真、负责、仔细，有良好的团队合作精神，良好的分析能力、沟通技巧。']
====
['风控数据分析师', '深圳白骑士大数据有限公司风控招聘', '10k-20k', '深圳', '经验1-3年', '本科及以上', '职位描述：职责描述：1、跟踪信贷审批政策与规则的执行效果，出具分析图表及报告，给出优化方案；2、监测贷前贷中贷后各关键环节风险指标，对风险预警异常情况进行深入分析，并提供相应的解决方案及政策建议；3、针对各逾期状态、风险敏感性客群进行深度挖掘分析；4、配合公司业务开展，给出风险评估报告并给出资产分配的优化建议。任职要求：1、本科及以上学历（985、211优先），应用数学、统计学、计算机科学、运筹学等相关专业；2、2年及以上消金行业风险数据分析相关工作经验，对贷前、贷中、贷后其中一个或多个环节的业务数据有较高的敏感度；3、熟练运用SQL、Python，熟悉Java；4、熟悉阿里云系列数据存储产品为加分项；5、有较强的

['产品经理（数据分析）', '八爪鱼产品部招聘', '15k-30k', '深圳', '经验3-5年', '本科及以上', '职位描述：岗位职责：1.产品规划与设计。负责数据分析产品（CEM客户体验管理）规划、业务流程设计、功能设计，制定阶段性目标；2.产品需求分析。制定产品需求计划，进行可行性分析及设计，撰写产品功能需求说明书，负责需求的跟踪和控制；3.竞品分析与用户反馈。把握市场前沿动态、研究并分析竞争对手、收集用户反馈，平衡业务发展和风控效率，保持产品竞争力；4.产品开发测试。主导产品的开发测试，协调各团队资源按期完成产品迭代；5.项目管理与推动。把握项目周期，跟踪产品的设计、开发、发布、运营等环节，确保项目按时完成和上线；6.数据分析与产品评估。定期对产品相关数据进行分析，善于从数据中发现问题，保持对数据的敏感度并指导行动。\xa0任职要求：1.数学、统计学、数量经济学相关专业，本科以上学历；2.熟练掌握数据分析、自然语言处理（NLP）基本方法论、2年以上互联网产品经验，有成功数据产品经验优先；3.具备产品宏观视角，能独立负责产品设计和规划，良好的需求分析及挖掘能力；4.能精准地把握客户需求，善于进行前瞻性的用户需求判断、引导及控制，并落实项目需求；5.具备较强的逻辑思维能力、文案写作能力，善于合作协调沟通，思维敏捷；6.良好的团队合作精神，出色的跨部门协调和问题解决能力，能承担较大的压力。']
====
['数据分析师', '音娱时光产品招聘', '10k-20k', '北京', '经验1-3年', '本科及以上', '职位描述：职责描述：1.\xa0\xa0\xa0\xa0\xa0完善业务数据监控体系，进行新功能指标埋点逻辑设计，搭建分析框架，使业务运营情况数据化呈现；2.\xa0\xa0\xa0\xa0\xa0深入了解业务，基于数据监控及分析研究，发现业务中的问题，同时能够解构问题、分析问题，给出有效改善的建议，并能输出分析报告；3.\xa0\xa0\xa0\xa0\xa0支持业务进行临时数据统计工作，并可从需求中提取业务目的，提高业务效率。4.\xa0\xa0\xa0\xa0\xa0与技术部门密切协作，确保统计需求准确落地以及线上统计无异常。岗位要求：1.\xa0\xa0\xa0\xa0\xa0本科及以上学历，数学、统计学、计算机科学等相关专业优先

['资深数据分析师', '微拍堂招聘', '25k-35k', '杭州', '经验3-5年', '大专及以上', '职位描述：工作职能：\xa01、构建业务数据分析体系，帮助确定各项业务数据指标；\xa0\xa02、负责监控项目上线效果情况，主动分析项目上线利弊点；3、通过数据分析对产品优化和运营活动提供可行性参考建议；\xa0职位要求：\xa01、统计学、应用数学、计算机等相关专业，本科及以上学历；\xa0\xa02、熟练掌握多种统计和挖掘方法，熟练使用HIVE查询数据、熟练使用SPSS、SAS、EXCEL等相关数据分析软件；\xa0\xa03、较强的数据敏感度，逻辑分析能力和文档写作能力；\xa0\xa04、有责任心，良好的沟通能力和组织管理能力以及心理承受能力，勇于接受挑战；\xa05、电商行业3-5年从业经验，熟悉行业规则，指标，有良好的行业资源']
====
['数据分析专家(J12055)', '哈啰出行招聘', '30k-50k', '上海', '经验3-5年', '本科及以上', '职位描述：工作职责:1.深入了解平台业务，建立基于业务场景的KPI指标体系，解构各类业务问题并有效以数据分析进行衡量；2.在遵循业务规则的前提下，根据数据分析结果，与业务部门合作制定出最佳的业务策略，完成产品、运营项目效果评估，并进行周期性数据报告拟写；3.基于行业情况展开相关调研和分析，产出相关趋势分析报告，助力相关业务决策；4.与团队内其他成员共同设计数据分析平台，建立数据分析的流程、规范和方法；5.快速响应，处理业务相关的临时需求，能够针对重点需求进行数据项目立项；任职资格:1、5年-8年工作经验，具有互联网行业经验者优先；本科以上学历，有大型互联网工作经验优先；2、具有良好的商业敏感度和数据分析技能，能够进行高效而实际的分析方法以解决各类的业务问题；3、熟悉数据挖掘的常用算法，拥有海量数据处理和挖掘经验者优先；4、具有较强的逻辑思维、清晰的沟通表达、项目管理及组织协调能力；5、具有优秀的团队合作精神；诚实、勤奋、严谨；有独立思考的能力。']
====
['数据分析师', '梦享网络招聘', '13k-25k', '深圳', '经验3-5年', '学历不限', '职位描述：岗位职责：1、负责产品海量数据的提取及分析挖掘，为相关部门提供数据支持。2、深入理解业务，协助